In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_path = os.path.abspath("..")

sys.path.append(project_path)

In [27]:
import json
from itertools import combinations
from urllib.parse import urlparse

from bs4 import BeautifulSoup as bs

from src.scrap_hrefs import get_units_hrefs
from src.scrap_infobox import extract_unit_data
from src.utils import fetch_page_content

In [4]:
aoe_wiki_url = "https://ageofempires.fandom.com/"
units_url = aoe_wiki_url + "wiki/Unit_(Age_of_Empires_III)"
unit_list_html = fetch_page_content(units_url)
href_soup = bs(unit_list_html, "html.parser")
main_page = href_soup.find('main', class_='page__main')


content = href_soup.find('div', class_='mw-parser-output')
h2s = content.find_all('h2', recursive=False)
sections = [h2.find('span', class_='mw-headline', recursive=False).text for h2 in h2s]
section_unit_hrefs = {section: get_units_hrefs(section, content) for section in sections}

In [5]:
# Units in more than one building
unique_units = {}
for key, values in section_unit_hrefs.items():
    unique_units[key] = set(values.keys())

# Generar todas las posibles parejas de llaves
key_pairs = combinations(unique_units.keys(), 2)

for k1, k2 in key_pairs:
    intersection = unique_units[k1] & unique_units[k2]
    if len(intersection) > 0:
        pair = (k1, k2)
        print(pair, " :", intersection)

('Explorer/War Chief/Monk/General', 'Town Center')  : {'Envoy'}
('Town Center', 'Trading Post/Native Embassy/Tambo')  : {'Chasqui'}
('Livestock Pen/Farm', 'Village')  : {'Goat'}
('Dock/Port', 'Revolutionary Variants')  : {'Fire Ship'}
('Dock/Port', 'Untrainable')  : {'Fire Ship'}
('Trading Post/Native Embassy/Tambo', 'Community Plaza')  : {'Healer'}
('Church/Mosque/Meeting House/Cathedral', 'Basilica')  : {'Priest', 'Spy'}
('Artillery Foundry', 'Factory')  : {'Rocket'}
('Kallanka', 'Community Plaza')  : {'Maceman'}
('Revolutionary Variants', 'Untrainable')  : {'Native Boy', 'Black Powder Wagon', 'Railroad Worker', 'White Jaguar', 'Great Cannon', 'White Wolf', 'Boneguard Swordsman', 'Great Plains Chief', 'Aztec Chief', 'Outlaw Musketeer', 'Boneguard Musketeer', "Lizzie's Flagship", 'Cherokee Horse Archer', 'Fierce Cougar', 'Dinghy', 'Cherokee Archer', 'Native American Chief', 'Buccaneer Captain', 'Horse', 'Flat-bottomed Boat', 'Miner', 'Treasure Ship', 'Greta', "Morgan's Flagship", 'Whi

In [6]:
all_hrefs = [href for sect in section_unit_hrefs.values() for href in sect.values()]

In [ ]:
item_types = {
    "text": [
        "Introduced in",
        "Required Home City Card",
        "Hit points",
        "Speed",
        "Line of Sight",
        "XP train bounty",
        "XP kill bounty",
        "Range",
        "Rate of Fire",
        "Train limit",
        "Ability",
        "Special ability",
        "Area of Effect",
        "Requires",
        "Regeneration",
        "Resource amount",
        "Gatherers",
        "Auto gather",
        "Pronunciation",
        "Garrison",
    ],
    "list": ["Type", "Civilization(s)", "Age", "Trained at", "Fatten rate"],
    "dict": ["Cost", "Train time", "Resistance", "Damage", "Bonus damage", "Fatten rate"],
    "ignore": ["Internal name"],
}

for url in all_hrefs:
    try:
        unit_html = fetch_page_content(url)
        unit_soup = bs(unit_html, "html.parser")
        fragment_id = urlparse(url).fragment
        if fragment_id:
            section = unit_soup.find(id=fragment_id)
            infobox = section.find_next("aside", class_="portable-infobox")
        else:
            infobox = unit_soup.find("aside", class_="portable-infobox")
        unit_data = extract_unit_data(infobox, item_types)
    except Exception as e:
        print(url)
        print(e)

https://ageofempires.fandom.com//wiki/Tiger#Axehilt_the_Tame_Tiger
'NoneType' object has no attribute 'find_next'
https://ageofempires.fandom.com//wiki/Eagle_Scout_(Age_of_Empires_III)
Information for label 'Bonus damage' could not be extracted in dict format.
Present values: '['Same as ranged attack']'
https://ageofempires.fandom.com//wiki/Dog#Peruvian_Dog
'NoneType' object has no attribute 'find_next'
https://ageofempires.fandom.com//wiki/White_Tiger_(Age_of_Empires_III)#Wagner_the_Pet_White_Tiger
Information for label 'Bonus damage' could not be extracted in dict format.
Present values: '['×1.3 vs. Light infantry']'
https://ageofempires.fandom.com//wiki/Bashibozuk
Information for label 'Bonus damage' could not be extracted in dict format.
Present values: '['×0.5 vs. Villager']'
https://ageofempires.fandom.com//wiki/Cougar#Buttercup
'NoneType' object has no attribute 'find_next'
https://ageofempires.fandom.com//wiki/Eclaireur
Information for label 'Bonus damage' could not be extracte